In [1]:
!pip install kaggle


In [2]:
from google.colab import files
files.upload()   # choose kaggle.json from your computer


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"halimaakhter","key":"6e7e4c254836ab2c243217292b7e81f9"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle competitions download -c house-prices-advanced-regression-techniques
!unzip house-prices-advanced-regression-techniques.zip -d house-prices-advanced-regression-techniques


  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 414MB/s]
Archive:  house-prices-advanced-regression-techniques.zip
  inflating: house-prices-advanced-regression-techniques/data_description.txt  
  inflating: house-prices-advanced-regression-techniques/sample_submission.csv  
  inflating: house-prices-advanced-regression-techniques/test.csv  
  inflating: house-prices-advanced-regression-techniques/train.csv  


In [5]:
!pip -q install lightgbm catboost optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 22.2 MB/s eta 0:00:00


In [6]:
import numpy as np, pandas as pd
from pathlib import Path
SEED = 42

train = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
test  = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")

print(train.shape, test.shape)
train.head(3)


(1460, 81) (1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [7]:
train_id = train["Id"].copy()
test_id  = test["Id"].copy()

y = np.log1p(train["SalePrice"])
X = train.drop(columns=["SalePrice", "Id"])
X_test_raw = test.drop(columns=["Id"])

full = pd.concat([X, X_test_raw], axis=0, ignore_index=True)


In [8]:
full = full.copy()

# ---- Indicator “has_*” features
full["HasBsmt"]   = (full["TotalBsmtSF"] > 0).astype(int)
full["HasGarage"] = full["GarageArea"].fillna(0).gt(0).astype(int)
full["HasPool"]   = full["PoolArea"].fillna(0).gt(0).astype(int)
full["Has2ndFlr"] = full["2ndFlrSF"].fillna(0).gt(0).astype(int)
full["HasFirepl"] = full["Fireplaces"].fillna(0).gt(0).astype(int)

# ---- Combined areas
full["TotalSF"]       = full["TotalBsmtSF"].fillna(0) + full["1stFlrSF"].fillna(0) + full["2ndFlrSF"].fillna(0)
full["TotalBath"]     = full["FullBath"].fillna(0) + 0.5*full["HalfBath"].fillna(0) + \
                        full["BsmtFullBath"].fillna(0) + 0.5*full["BsmtHalfBath"].fillna(0)
full["TotalPorchSF"]  = full[["OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","WoodDeckSF"]].fillna(0).sum(axis=1)

# ---- Ages (careful with missing GarageYrBlt)
full["HouseAge"]   = full["YrSold"] - full["YearBuilt"]
full["RemodAge"]   = full["YrSold"] - full["YearRemodAdd"]
full["GarageYrBlt"] = full["GarageYrBlt"].fillna(full["YearBuilt"])
full["GarageAge"]  = (full["YrSold"] - full["GarageYrBlt"]).clip(lower=0)

# ---- Neighborhood median LotFrontage impute (classic trick)
lf_med = full.groupby("Neighborhood")["LotFrontage"].transform("median")
full["LotFrontage"] = full["LotFrontage"].fillna(lf_med).fillna(full["LotFrontage"].median())

# ---- Ordinal encode quality/condition fields
ord_map_qual = {"Po":1, "Fa":2, "TA":3, "Gd":4, "Ex":5}
for col in ["ExterQual","ExterCond","BsmtQual","BsmtCond","HeatingQC","KitchenQual","FireplaceQu",
            "GarageQual","GarageCond","PoolQC"]:
    if col in full:
        full[col] = full[col].map(ord_map_qual).fillna(0).astype(int)

# Basement exposure (ordered)
ord_bsmtexp = {"No":1, "Mn":2, "Av":3, "Gd":4}
if "BsmtExposure" in full: full["BsmtExposure"] = full["BsmtExposure"].map(ord_bsmtexp).fillna(0).astype(int)

# Basement finish types (ordered)
ord_bsmttype = {"Unf":1, "LwQ":2, "Rec":3, "BLQ":4, "ALQ":5, "GLQ":6}
for col in ["BsmtFinType1","BsmtFinType2"]:
    if col in full: full[col] = full[col].map(ord_bsmttype).fillna(0).astype(int)

# Functional (ordered with Typ best)
ord_func = {"Sal":1,"Sev":2,"Maj2":3,"Maj1":4,"Mod":5,"Min2":6,"Min1":7,"Typ":8}
if "Functional" in full: full["Functional"] = full["Functional"].map(ord_func).fillna(ord_func["Typ"]).astype(int)

# ---- Fill remaining numeric/categorical missing
num_cols = full.select_dtypes(include=[np.number]).columns
cat_cols = full.select_dtypes(exclude=[np.number]).columns

full[num_cols] = full[num_cols].fillna(0)
full[cat_cols] = full[cat_cols].fillna("None")


In [9]:
from scipy.stats import skew

skewed = full[num_cols].apply(lambda s: skew(s.dropna())).abs()
to_fix = skewed[skewed > 0.75].index.tolist()

for c in to_fix:
    full[c] = np.log1p(full[c].clip(lower=0))


In [10]:
full_enc = pd.get_dummies(full, columns=cat_cols, drop_first=True)

X_proc = full_enc.iloc[:len(X), :].reset_index(drop=True)
X_test = full_enc.iloc[len(X):, :].reset_index(drop=True)

X_proc.shape, X_test.shape


((1460, 228), (1459, 228))

In [12]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


oof = np.zeros(len(X_proc))
test_pred = np.zeros(len(X_test))
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

# Train on log target; evaluate on original scale
for fold, (tr, va) in enumerate(kf.split(X_proc), 1):
    X_tr, X_va = X_proc.iloc[tr], X_proc.iloc[va]
    y_tr, y_va = y.iloc[tr], y.iloc[va]

    model = lgb.LGBMRegressor(
        n_estimators=5000,
        learning_rate=0.03,
        num_leaves=31,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=1,
        reg_alpha=0.1,
        reg_lambda=0.2,
        random_state=SEED,
        objective="rmse"
    )

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="rmse",
        callbacks=[lgb.early_stopping(200), lgb.log_evaluation(200)]
    )

    va_pred_log = model.predict(X_va, num_iteration=model.best_iteration_)
    va_pred = np.expm1(va_pred_log)
    oof[va] = va_pred

    test_pred += np.expm1(model.predict(X_test, num_iteration=model.best_iteration_)) / kf.n_splits

cv_score = rmse(np.expm1(y), oof)
print(f"CV RMSE (on original SalePrice): {cv_score:,.4f}")


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001187 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=tru

KeyboardInterrupt: 

In [ ]:
best_iter = 4000  # fallback; optionally set to model.best_iteration_ from the last fold
final_model = lgb.LGBMRegressor(
    n_estimators=best_iter,
    learning_rate=0.03,
    num_leaves=31,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    reg_alpha=0.1,
    reg_lambda=0.2,
    random_state=SEED,
    objective="rmse"
)
final_model.fit(X_proc, y)

final_pred = np.expm1(final_model.predict(X_test))
sub = pd.DataFrame({"Id": test_id, "SalePrice": final_pred})
sub.to_csv("submission.csv", index=False)
sub.head()


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001208 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=tru

,Id,SalePrice
0,1461,118871.389462
1,1462,164697.936259
2,1463,183547.430745
3,1464,196829.127507
4,1465,181289.890904


In [ ]:
from google.colab import files
files.download("submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
from catboost import CatBoostRegressor, Pool

# Start again from raw X / X_test_raw (before our one-hot step)
X_cat = pd.concat([X, X_test_raw], axis=0, ignore_index=True).copy()

# Minimal impute
for c in X_cat.select_dtypes(include=[np.number]).columns:
    X_cat[c] = X_cat[c].fillna(0)
for c in X_cat.select_dtypes(exclude=[np.number]).columns:
    X_cat[c] = X_cat[c].fillna("None")

cat_cols = X_cat.select_dtypes(exclude=[np.number]).columns.tolist()
Xc = X_cat.iloc[:len(X), :].reset_index(drop=True)
Tc = X_cat.iloc[len(X):, :].reset_index(drop=True)

oof_cb = np.zeros(len(Xc))
test_cb = np.zeros(len(Tc))
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

for fold, (tr, va) in enumerate(kf.split(Xc), 1):
    model = CatBoostRegressor(
        iterations=6000,
        depth=8,
        learning_rate=0.03,
        l2_leaf_reg=3.0,
        loss_function="RMSE",
        random_state=SEED,
        verbose=500
    )
    model.fit(
        Xc.iloc[tr], y.iloc[tr],
        eval_set=(Xc.iloc[va], y.iloc[va]),
        cat_features=cat_cols,
        use_best_model=True
    )
    oof_cb[va] = np.expm1(model.predict(Xc.iloc[va]))
    test_cb += np.expm1(model.predict(Tc)) / kf.n_splits

print("CatBoost CV RMSE:", rmse(np.expm1(y), oof_cb))

sub_cb = pd.DataFrame({"Id": test_id, "SalePrice": test_cb})
sub_cb.to_csv("submission_catboost.csv", index=False)


0:	learn: 0.3831058	test: 0.4256138	best: 0.4256138 (0)	total: 139ms	remaining: 13m 52s
500:	learn: 0.0671453	test: 0.1343048	best: 0.1343048 (500)	total: 44.2s	remaining: 8m 4s
1000:	learn: 0.0435073	test: 0.1317534	best: 0.1317534 (1000)	total: 1m 33s	remaining: 7m 47s
1500:	learn: 0.0278881	test: 0.1310068	best: 0.1309963 (1499)	total: 2m 21s	remaining: 7m 3s
2000:	learn: 0.0188120	test: 0.1308306	best: 0.1308127 (1944)	total: 3m 9s	remaining: 6m 18s
2500:	learn: 0.0129817	test: 0.1307782	best: 0.1307461 (2086)	total: 3m 57s	remaining: 5m 32s
3000:	learn: 0.0092057	test: 0.1307828	best: 0.1307461 (2086)	total: 4m 44s	remaining: 4m 44s
3500:	learn: 0.0067260	test: 0.1307992	best: 0.1307461 (2086)	total: 5m 33s	remaining: 3m 57s
4000:	learn: 0.0049816	test: 0.1308043	best: 0.1307461 (2086)	total: 6m 31s	remaining: 3m 15s
4500:	learn: 0.0037243	test: 0.1308184	best: 0.1307461 (2086)	total: 7m 18s	remaining: 2m 25s
5000:	learn: 0.0028685	test: 0.1308262	best: 0.1307461 (2086)	total: 8m 

In [14]:
from google.colab import files
files.download("submission_catboost.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>